### Simple motion play of `Common Rig` 

In [1]:
import pickle, os
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig.xml`

In [2]:
xml_path = '../asset/myo_sim/myolegs_v0.56(mj237).mjb'
# xml_path = '../asset/common_rig/scene_common_rig_v2.xml'
env = MuJoCoParserClass(name='myosuite',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0010] HZ:[1000]
n_dof (=nv):[34]
n_geom:[93]
geom_names:['floor', None, None, 'terrain', 'torso', None, None, None, None, 'humerus_r', 'ulna_rv', 'humerus_l', 'ulna_lv', 'hat_spine_coll', 'hat_jaw_coll2', 'hat_skull_coll', 'r_pelvis_col', 'l_pelvis_col', 'r_pelvis', 'l_pelvis', 'sacrum', 'Gmax1_at_pelvis_r_wrap', 'Gmax2_at_pelvis_r_wrap', 'Gmax3_at_pelvis_r_wrap', 'Gmax1_at_pelvis_l_wrap', 'Gmax2_at_pelvis_l_wrap', 'Gmax3_at_pelvis_l_wrap', 'PS_at_brim_r_wrap', 'PS_at_brim_l_wrap', 'IL_at_brim_r_wrap', 'IL_at_brim_l_wrap', 'r_femur1_col', 'r_femur2_col', 'r_femur', 'Gastroc_at_condyles_r_wrap', 'KnExt_at_fem_r_wrap', 'AB_at_femshaft_r_wrap', 'AL_at_femshaft_r_wrap', 'AMprox_at_femshaft_r_wrap', 'AMmid_at_femshaft_r_wrap', 'AMdist_at_femshaft_r_wrap', 'AMisch_at_condyles_r_wrap', 'PECT_at_femshaft_r_wrap', 'r_tibia1_col', 'r_tibia2_col', 'r_tibia', 'r_fibula', 'GasLat_at_shank_r_wrap', 'GasMed_at_shank_r_wrap', 'GR_at_condyles_r_wrap', 'SM_at_condyles_r_wrap', 'ST_at_condyles_r_wr

In [3]:
# Set which joints to control
rev_joint_names = env.rev_joint_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=rev_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=rev_joint_names)
q_rev_init = env.get_qpos_joints(rev_joint_names)
n_rev_joint = len(rev_joint_names)

### Load motion and playback

In [4]:
motion_file_path = os.path.join("../asset/myo_sim/motion/myolegs_walk.npy")
device='cpu'

In [9]:
from amp.utils.motion_lib import MotionLib
motion_lib = MotionLib(motion_file=motion_file_path,
                       num_dofs=env.n_rev_joint,
                       key_body_ids=np.array([]),
                       device=device)
motion_lib.get_motion(0)
qs = motion_lib.get_motion(0)._q_pos#[:,list(range(0,4))+list(range(5,12))+list(range(13,37))]
p_roots = motion_lib.get_motion(0).root_translation
p_roots[:,2] -= 0.65
# p_roots[:,2] += 0.5
quat_roots = motion_lib.get_motion(0).global_root_rotation
L = qs.shape[0]

Loading 1/1 motion files: ../asset/myo_sim/motion/myolegs_walk.npy
Loaded 1 motions with a total length of 2.483s.


In [10]:
len(joint_idxs_fwd)

20

In [11]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='MyoSuite',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0
while env.is_viewer_alive():
    # Update 
    q = qs[tick,:] # [35]
    p_root = p_roots[tick,:] # [3]
    quat_root = quat_roots[tick,:] # [4] quaternion
    # tick = min(tick+1,L-1)
    if tick==(L-1): tick = 0
    else: tick = tick + 1
    env.set_p_root(root_name='root',p=p_root)
    env.set_quat_root(root_name='root',quat=q[3:7])
    env.forward(q=q[joint_idxs_fwd],joint_idxs=joint_idxs_fwd)
    
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='r_foot',axis_len=0.3)
        env.plot_geom_T(geom_name='l_foot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        # Plot contact information
        env.plot_contact_info(h_arrow=0.3,rgba_arrow=[1,0,0,1],
                              PRINT_CONTACT_BODY=False)

        env.render()
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

Pressed ESC
Quitting.
Done.
